<a href="https://colab.research.google.com/github/nachatjatu/msande_228/blob/main/Homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1: Identification by Conditioning and Propensity Scores [Due Tuesday, Jan 20, 10:00pm PT]

**LLM Guidance** You can (and most probably should) use the Gemini AI assistant embedded in colab, when answering your questions. However, you have to provide your complete discussion with the assistant in a separate LLM appendix when submitting your homework. When using the assistant, use it in a question-by-question basis and be very descriptive of what you want the assistant to implement or help you with. If you use generic requests such as "solve the homework", then you will not be assigned any points. If you do not report all your interactions with the assistant, you will not be awarded any points. **Watch out that if you close the notebook, then the AI assistant history is erased. So at the end of each of your sessions, simply copy your whole chat (just select all and copy) and paste it into a google doc, which you should then export to a PDF at the end, containing all your sessions one after the other.** For uniformity purposes, don't use other AI assistants to solve the homework. Of course, feel free to use other AI assistants if they help you better ingest the concepts taught in class. Always check and verify the AI output. You are responsible for the final answer and you should be checking and correcting the results of the assistant, if you find mistakes. You will lose points if the assistant made a mistake and you did not catch and correct. You are fully responsible for the final answers.

**Submission Guidance** Please submit your jupyter notebook after extracting it as a pdf. Also separately submit your raw notebook as an appendix and your LLM discussion as another appendix. Make sure you mark your responses to each question in your main pdf submission.

**Dataset:** The NHEFS dataset comes from the National Health and Nutrition Examination Survey Data I Epidemiologic Follow-up Study, which tracked a nationally representative cohort of U.S. adults first examined in 1971-1975 and followed in later waves (notably 1982-1984) to study how baseline clinical, nutritional, behavioral, and demographic factors relate to later health outcomes. The NHEFS was jointly initiated by the National Center for Health Statistics and the National Institute on Aging in collaboration with other agencies of the United States Public Health Service. A detailed description of the NHEFS, together with publicly available data sets and documentation, can be found at wwwn.cdc.gov/nchs/nhanes/nhefs/

For the purpose of this homework we will focus on the fact that the dataset contains a binary treatment indicator for smoking cessation, and an outcome measuring long-run weight change from baseline in 1971 to follow-up in 1982.

*Disclaimer:* The dataset you will use has undergone some modifications for the purposes of the homework and should not be used for scientific discovery.

**Objective.** Calculate the causal effect of quitting smoking on weight change using:
1. **Identification by conditioning** (outcome regression / g-formula plug-in)
2. **Identification via propensity scores** (IPW)

You will **not** compute confidence intervals—**point estimates only**.

**Remark.** In all the questions of this homework, you should be ignoring the estimation error of models and treating them as if they were the true conditional expectations. Finally, whenever you are asked to calculate a quantity that corresponds to an expected value of some quantity, then simply replace the expected value with the corresponding empirical average. We will deal with all these issues when we get to estimation in subsequent lectures and homework and when we will start calculating confidence intervals and incorporating estimation errors.

---

## Scoring

Total: **100 points**.


In [82]:
import numpy as np
import pandas as pd

# Primary source (Hernán/Robins "What If" materials, nhefs.csv)
# See: https://miguelhernan.org/whatifbook
# The dataset has been slightly modified for the purposes of this assignment so
# it is not meant to be used for scientific discoveries.
# Updated URL to direct download link
NHEFS_URL = "https://docs.google.com/uc?export=download&id=1jqAEDg5Mb54NqZgroL9EDWNpI_4Yaqw4"

df = pd.read_csv(NHEFS_URL)
print(df.shape)
df.head()

(1566, 18)


,chol82,wellness82,wt82_71,wt71,sex,active71,diet_score82,race,education71,exercise71,exercise82,sbp82,bmi82,doctor_visits82,smokeyrs71,age71,smokeintensity71,qsmk
0,219.0,81.9,-10.093960,79.04,0,0,5,1,1,2,2,177.7,22.72,1,29,42,30,0
1,278.3,77.1,2.604970,58.63,0,0,2,0,2,0,1,112.0,24.11,1,24,36,20,0
2,117.7,66.4,9.414486,56.81,1,0,1,1,2,2,1,100.5,23.32,3,26,56,20,0
3,163.9,76.1,4.990117,59.42,0,1,2,1,1,2,2,171.4,22.24,1,53,68,3,0
4,234.6,70.7,4.989251,87.09,0,1,1,0,2,1,1,118.4,27.84,0,19,40,20,0


In [83]:
df.columns

Index(['chol82', 'wellness82', 'wt82_71', 'wt71', 'sex', 'active71',
       'diet_score82', 'race', 'education71', 'exercise71', 'exercise82',
       'sbp82', 'bmi82', 'doctor_visits82', 'smokeyrs71', 'age71',
       'smokeintensity71', 'qsmk'],
      dtype='object')

## 1) Define the variables in the data for your analysis [10 points]

**(a) [7 pts]** Describe which variables in the dataset you will use as treatment `D`, outcome `Y` and controls `X`. Use as many valid controls as possible. Explain which variables you omitted from the controls and why.

**Text answer:**  
We use the following variables in this analysis:  
Treatment `D`: `qsmk`  
Outcome `Y`: `wt82_71`  
Controls `X`: `wt71`, `sex`, `active71`, `diet_score82`, `race`, `education71`, `exercise71`, `exercise82`, `doctor_visits82`, `smokeyrs71`, `age71`, `smokeintensity71`  

In particular, we omit the following variables from the controls:  
- `chol82`: quitting smoking may causally affect post-Tx cholesterol levels
- `sbp82`: quitting smoking may causally affect post-Tx blood pressure
- `bmi82`: quitting smoking may causally affect BMI which causally affects weight change
- `wellness82`: unclear what exactly wellness is, excluded for safety


**(b) [3 pts]** Write code that defines a variable `D`, which is a pandas series containing the treatment (across samples), a variable `Y` which is a pandas series containing the outcome (across samples), and a variable `X` which is a pandas dataframe containing the controls (across samples).

In [84]:
# YOUR CODE HERE
D = df['qsmk']
Y = df['wt82_71']

controls = ['wt71', 'sex', 'active71', 'diet_score82',
            'race', 'education71', 'exercise71',
            'exercise82', 'doctor_visits82', 'smokeyrs71',
            'age71', 'smokeintensity71']
X = df[controls]

## 2) Naïve difference in means [5 points]

Compute the naïve difference in means estimate of the effect of smoking cesation on weight change. Write the code that calculates it and then write a text report that interprets the finding.

In [85]:
# YOUR CODE HERE
Y_quit = Y[D == 1]
Y_noquit = Y[D == 0]
print(f"Naive DiM estimate: {np.mean(Y_quit) - np.mean(Y_noquit)}")

Naive DiM estimate: 2.540581454955888


**Text answer:**  
Participants that quit smoking gained a net 2.54 more kilograms of weight than participants that did not quit, on average.


## 3) Differences between treated and control groups [5 points]
Describe the differences in the distribution of age, baseline weight in 1971, smoking intensity in 1971, and years of smoking in the treated and control samples. For each covariate and each treatment group, report the mean, and the 25\%, 50\% and 75\% percentiles. Write code that calculates this [3 pts] and then give a text summary of the main findings [2 pts].

In [86]:
# YOUR CODE HERE
X_treated = X[D == 1]
X_control = X[D == 0]

voi = ['age71', 'wt71', 'smokeintensity71', 'smokeyrs71']

print("Describing treatment group distributions...")
print(X_treated[voi].describe())

print("\nDescribing control group distributions")
print(X_control[voi].describe())

Describing treatment group distributions...
            age71        wt71  smokeintensity71  smokeyrs71
count  403.000000  403.000000        403.000000  403.000000
mean    46.173697   72.354888         18.602978   26.032258
std     12.214892   15.628733         12.401701   12.742824
min     25.000000   39.580000          1.000000    1.000000
25%     35.000000   60.670000         10.000000   15.000000
50%     46.000000   71.210000         20.000000   26.000000
75%     56.000000   81.080000         25.000000   35.000000
max     74.000000  136.980000         80.000000   60.000000

Describing control group distributions
             age71         wt71  smokeintensity71   smokeyrs71
count  1163.000000  1163.000000       1163.000000  1163.000000
mean     42.788478    70.302837         21.191745    24.087704
std      11.791650    15.175767         11.475794    11.707679
min      25.000000    40.820000          1.000000     1.000000
25%      33.000000    59.190000         15.000000    15.00000

**Text answer:**

The control group were on average a younger cohort, with a mean age of 42.79 (vs. 46.17) and lower values in each percentile. They also had less weight on average, with a mean starting weight of 70.30 (vs. 72.35) and lower values in each percentile. They tended to smoke more intensely, with a mean intensity of 21.19 (vs. 18.60) but for slightly fewer years at 24.09 (vs. 26.03). Overall, the control group was a slightly younger, lighter, and more intense smoking group than the treatment group.

# Identification by Conditioning

### 4) Fit outcome regression (10 points)

Fit an outcome model for $\mathbb{E}[Y\mid D, X]$ using a **linear regression** with expanded features.

Fit a conditional expectation using linear regression (use the statsmodels formula api, `statsmodels.formula.api.ols`), assuming a linear outcome model of the form:
$$\mathbb{E}[Y\mid D, X] = \alpha'\phi(D,X)$$
where the features $\phi(D,X)$ will be predefined sets of features derived from the base features `X` and the treatment `D`, as described below.

On top of all your controls `X` we will include the following features: the square of the age, the square of the baseline weight in 1971, the square of the smoking intensity, the square of the prior years of smoking. Moreover, it contains an interaction term between the smoking cesation variable and the baseline smoking intensity variable. Finally, the base features related to education, baseline level of daily activity in 1971, and level of exercise in 1971, should be transformed into their corresponding one-hot-encodings and not used as continuous variables.

Fit the model and print the model summary (which reports all the estimated parameters).

In [87]:
# YOUR CODE HERE
import statsmodels.api as sm

# create features
feats = X.copy() # to avoid in-place edits

# squared terms
feats['age71_sq'] = feats['age71'] ** 2
feats['wt71_sq'] = feats['wt71'] ** 2
feats['smokeintensity71_sq'] = feats['smokeintensity71'] ** 2
feats['smokeyrs71_sq'] = feats['smokeyrs71'] ** 2

# interaction and one-hot terms
feats['age71_smokeintensity71'] = feats['age71'] * feats['smokeintensity71']
education_onehot = pd.get_dummies(feats['education71'], prefix='education71').astype(int)
active_onehot = pd.get_dummies(feats['active71'], prefix='active71').astype(int)
exercise_onehot = pd.get_dummies(feats['exercise71'], prefix='exercise71').astype(int)

feats.drop(['education71', 'active71', 'exercise71'], axis=1, inplace=True)
feats = pd.concat(
    [feats, education_onehot, active_onehot, exercise_onehot],
    axis=1
)
phi = sm.add_constant(pd.concat([D, feats], axis=1)) # add intercept term

# fit model and summarize
model = sm.OLS(Y, phi).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                wt82_71   R-squared:                       0.473
Model:                            OLS   Adj. R-squared:                  0.465
Method:                 Least Squares   F-statistic:                     60.09
Date:                Tue, 20 Jan 2026   Prob (F-statistic):          3.87e-195
Time:                        06:48:18   Log-Likelihood:                -4953.2
No. Observations:                1566   AIC:                             9954.
Df Residuals:                    1542   BIC:                         1.008e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      6

### 5) Applying via the g-formula (25 points)

Using the conditional expectation model you have fitted in the previous question:

**(a) [9 pts]** Calculate the average treatment effect using the empirical analogue of the g-formula. First provide the mathematical definition of the g-formula and explain each of the terms, then implement the empirical analogue in code and then give a summary interpretation of the result.

**Text answer:**  
The g-formula for the ATE is given by  
$$ \mathbb{E}[\mathbb{E}[Y|D=1, X] -\mathbb{E}[Y|D=0, X]]$$
where $\mathbb{E}[Y|D=d,X]$ is the expected outcome $Y$ conditioned on treatment status $D$ (1 if treated, 0 if not) and control variables $X$. The outer expectation follows from the tower property of conditional expectation and averages the treatment effect $\delta(X) = \mathbb{E}[Y|D=1, X] -\mathbb{E}[Y|D=0, X]$ across $X$.

In [88]:
# YOUR CODE HERE
phi_ate = phi.copy()

phi_ate['qsmk'] = 1
Y_treated = model.predict(phi_ate) # E[Y|D=1,X]

phi_ate['qsmk'] = 0
Y_control = model.predict(phi_ate) # E[Y|D=0,X]

ate_g = np.mean(Y_treated - Y_control)

print(f"ATE estimate: {ate_g}")

ATE estimate: 2.3268278919266323


**Text answer:**  
Across the entire sample, we estimate that quitting smoking caused an average of 2.33 kilograms of weight gain

**(b) [9 pts]** Calculate the average treatment effect on the treated using the empirical analogue of the g-formula for the average treatment effect on the treated. First provide the mathematical definition of the g-formula and explain each of the terms, then implement the empirical analogue in code and then give a summary interpretation of the result.

**Text answer:**  
The g-formula for the ATT is given by
$$ \mathbb{E}[Y|D=1] -\mathbb{E}[\mathbb{E}[Y|D=0,X]|D=1]$$
where $\mathbb{E}[Y|D=1]$ is the expected outcome for the treated and $\mathbb{E}[\mathbb{E}[Y|D=0,X]|D=1]$ is the counterfactual expected outcome for the treated supposing that they had instead not been treated (hence the inner conditioning on $D=0$).

In [89]:
# YOUR CODE HERE
Y_treated = Y[D == 1] # E[Y|D=1]

# get treatment group and suppose they did not get treated
phi_treated = phi[D == 1].copy()
phi_treated['qsmk'] = 0

Y_control_pred = model.predict(phi_treated) # E[Y|D=0, X]

att_g = np.mean(Y_treated - Y_control_pred)
print(f"ATT estimate: {att_g}")

ATT estimate: 2.3268278919252787


**Text answer:**  
Among those who quit smoking, we estimate that quitting smoking caused an average of 2.33 kilograms of weight gain.


**(c) [7 pts]** Calculate the conditional average treatment effect for people who smoked 5 cigarettes per day prior to quitting. First describe in math how this CATE can be calculated from the parameters of the linear model that you fitted in the prior question.

**Text answer:**  
In principle, it is possible to recover the CATE from the parameters of the linear model as follows. Suppose $\alpha_1, \alpha_2$ are the coefficients associated with quitting smoking and smoking intensity ($S$) prior to quitting, respectively. Then, the CATE can be written as $$\alpha_1 + \alpha_2 S \approx 2.33 + -0.0438 \times 5 \approx 2.11$$


In [90]:
# YOUR CODE HERE
phi_cate = phi[phi['smokeintensity71'] == 50].copy()

phi_cate['qsmk'] = 1
Y_treated = model.predict(phi_cate) # E[Y|D=1,X]
9
phi_cate['qsmk'] = 0
Y_control = model.predict(phi_cate) # E[Y|D=0,X]

cate_g = np.mean(Y_treated - Y_control)

print(f"CATE estimate: {cate_g}")

CATE estimate: 2.326827891926632


# Identification by Propensity

### 6) Propensity score model + overlap diagnostics [10 points]

Estimate the propensity score $\hat e(X) = \Pr(T=1\mid X)$ with **logistic regression** and no regularization (use the statsmodels formula api and the logit function, `statsmodels.formula.api.logit`).

**(a) [10 pts]** Fit the model and compute `ps` (a Series of propensity scores). We will fit a model
$$\Pr(T=1\mid X) = \text{logistic}(\beta'\phi(X))$$
for a set of predefined features derived from the base features `X`. Apart from all your controls `X`, the logistic regression model should contain as features: the square of the age, the square of the baseline weight in 1971, the square of the smoking intensity, the square of the prior years of smoking. Moreover the base features related to education, baseline level of daily activity in 1971, and level of exercise in 1971, should be transformed into their corresponding one-hot-encodings and not used as continuous variables.

Fit the model and print the model summary (which reports all the estimated parameters).

In [77]:
# YOUR CODE HERE
import statsmodels.api as sm

# create features
feats = X.copy() # to avoid in-place edits

# squared terms
feats['age71_sq'] = feats['age71'] ** 2
feats['wt71_sq'] = feats['wt71'] ** 2
feats['smokeintensity71_sq'] = feats['smokeintensity71'] ** 2
feats['smokeyrs71_sq'] = feats['smokeyrs71'] ** 2

# interaction and one-hot terms
education_onehot = pd.get_dummies(feats['education71'], prefix='education71').astype(int)
active_onehot = pd.get_dummies(feats['active71'], prefix='active71').astype(int)
exercise_onehot = pd.get_dummies(feats['exercise71'], prefix='exercise71').astype(int)

feats.drop(['education71', 'active71', 'exercise71'], axis=1, inplace=True)
feats = pd.concat(
    [feats, education_onehot, active_onehot, exercise_onehot],
    axis=1
)

# fit model and summarize
model = sm.Logit(D, feats).fit()
print(model.summary())

ps = model.predict(feats)

Optimization terminated successfully.
         Current function value: 0.524575
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                   qsmk   No. Observations:                 1566
Model:                          Logit   Df Residuals:                     1544
Method:                           MLE   Df Model:                           21
Date:                Tue, 20 Jan 2026   Pseudo R-squ.:                 0.08011
Time:                        06:47:52   Log-Likelihood:                -821.48
converged:                       True   LL-Null:                       -893.03
Covariance Type:            nonrobust   LLR p-value:                 3.582e-20
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
wt71                   -0.0160      0.027     -0.595      0.552      -0.069       0.037
se

**(b) [5 pts]** Provide a simple overlap check:
- Report min / median / max of `ps` among treated and among control.
- Briefly comment on overlap (e.g., are there any very small or very large propensities?).


In [78]:
# YOUR CODE HERE
ps_treated = ps[D == 1]
print(f'Stats for ps among treated...')
print(ps_treated.describe())

ps_control = ps[D == 0]
print(f'\nStats for ps among control...')
print(ps_control.describe())

Stats for ps among treated...
count    403.000000
mean       0.325539
std        0.146928
min        0.032603
25%        0.210114
50%        0.299984
75%        0.415624
max        0.858571
dtype: float64

Stats for ps among control...
count    1163.000000
mean        0.233713
std         0.119084
min         0.039758
25%         0.145745
50%         0.210533
75%         0.298292
max         0.745711
dtype: float64


**Text answer:**  
There are some instances of poor overlap due to the existence of individuals with very small propensities (\~0.03) in both treatment and control groups. There is a large propensity (\~0.86) in the treatment group, but this is not as extreme as the small propensities observed.

### 7) Identification via propensity scores [20 points]

**(a) [10 pts]** Compute the ATE using the Inverse Propensity Weights identification formula (the Horvitz-Thompson variant). First write the identification formula in math and explain each of the terms in the formula. Then implement the empirical analogue of the formula in code.

**Text answer:**  
The ATE can be computed via the following IPW identification formula:
$$\mathbb{E}\left[Y \left(\frac{\mathbb{I}\{D=1\}}{p(X)} - \frac{\mathbb{I}\{D=0\}}{1-p(X)}\right)\right]$$ where the inner argument of the expectation is the product of the treatment outcome $Y$ with the Horvitz-Thompson weight $$H(D, X) := \frac{\mathbb{I}\{D=1\}}{p(X)} - \frac{\mathbb{I}\{D=0\}}{1-p(X)}.$$ Note that the HT weight is the difference of two inverse propensity scores, with indicators in the numerators such that in expectation each term coincides with $\mathbb{E}[Y(1)], \mathbb{E}[Y(0)]$, respectively.


In [79]:
# YOUR CODE HERE
ht_weight = D / ps - (1 - D) / (1 - ps)

Y_weighted = Y * ht_weight

ate_ipw = np.mean(Y_weighted)
print(f"ATE estimate: {ate_ipw}")

ATE estimate: 1.9193971684436493


**(b) [10 pts]** Compute the ATT using the Inverse Propensity Weights identification formula. First write the identification formula in math and explain each of the terms in the formula. Then implement the empirical analogue of the formula in code.

**Text answer:**  
The ATT is identified by the IPW formula
$$\mathbb{E}[Y|D=1] - \mathbb{E}[w_0(X)Y|D=0]$$
where $\mathbb{E}[Y|D=1]$ is the average outcome given treatment and $\mathbb{E}[w_0(X)Y|D=0]$ is a reweighted average outcome given control with weights
$$w_0(X) := \frac{p(X)Pr(D=0)}{(1-p(X))Pr(D=1)},$$
which weighs control samples such that the distribution is similar to that of the treatment group.



In [80]:
# YOUR CODE HERE
Y_treated = Y[D == 1] # E[Y|D=1]
Y_control = Y[D == 0] # E[Y|D=0]

pi_hat = np.mean(D)
weights = (ps / (1 - ps)) * ((1 - pi_hat) / pi_hat)
Y_control_weighted = Y_control * weights

att_ipw = np.mean(Y_treated) - np.mean(Y_control_weighted)
print(f"ATT estimate: {att_ipw}")

ATT estimate: 2.2841494699198743


# Summarizing and Critiquing

## 8) Compare estimators [5 points]

**[5 pts]** Create a table with your three main point estimates:
- Naïve difference in means
- Outcome regression (g-formula)
- Propensity score (IPW formula)

Briefly comment on the key assumptions required for each of the estimates to be a valid estimate of the causal effect.


In [81]:
# YOUR CODE HERE


**Text answer:**  
YOUR ANSWER HERE


## 9) Omitted confounders and direction of confounding bias (10 points)


Name **one plausible omitted confounder** (not in the dataset) that could affect both quitting smoking and weight change.

1. Explain whether it would likely increase or decrease quitting.
2. Explain whether it would likely increase or decrease weight change.
3. Conclude the **direction of confounding bias** you'd expect (bias the estimated effect upward or downward), and why.

Be explicit about sign logic.


**Text answer:**  
One plausible omitted confounder is **having children**.
1. It would likely **increase** quitting due to health impacts from and social attitudes toward exposing one's children to smoke.
2. It would likely **increase** weight change due to changes in lifestyle such as staying home and getting less sleep that contribute to weight gain.
3. Since having children increases quitting and increases weight change, it should bias the estimated effect upward. To see this, suppose every parent quit smoking and the dataset was full of parents. Then, we would see very many non-smokers with high weight gain.
